In [1]:
import pandas as pd
import requests as r
import matplotlib.pyplot as plt
import plotly.express as px
import plotly.graph_objs as go
import numpy as np
from bs4 import BeautifulSoup
from matplotlib.pyplot import xlabel
import os


In [2]:
plot_folder = '/Users/eddie/cs163/src/Plots'
data_folder = '/Users/eddie/cs163/src/Data'

# Data Collection
## CPI Data Import

In [4]:
url = 'https://api.bls.gov/publicAPI/v2/timeseries/data/'

data = {
    "seriesid": ["PCU311412311412", "PCU311423311423", "PCU3117--3117--", "PCU31191-31191-", "PCU311991311991", "PCU445---445---", "CUSR0000SAF"],
    "startyear": "2005",
    "endyear": "2024",
    "catalog": True,
    "calculations": False,
    "annualaverage": False,
    "aspects": False,
    "registrationkey": "5974f3d1e0ac4efdad23d2e1b3e4b4e3"
}

columns_BLS = ['Frozen Food Manufacturing PPI', 'Dried and dehydrated Food Manufacturing PPI', 'Seafood Production and Packaging PPI', 'Snack Food Manufacturing PPI', 'Perishable Food Manufacturing PPI', 'Food and Beverages Retailers PPI', 'Food and Beverages CPI (target)']
html = r.post(url, json=data)
html = html.json()

grid = []
month_to_num = {
    'January': 1,
    'February': 2,
    'March': 3,
    'April': 4,
    'May': 5,
    'June': 6,
    'July': 7,
    'August': 8,
    'September': 9,
    'October': 10,
    'November': 11,
    'December': 12
}

for table_index in range(0, len(columns_BLS)) :
    data = html['Results']['series'][table_index]['data']
    tmp_df = pd.DataFrame(data, columns=['year', 'periodName', 'value'])
    tmp_df.periodName = tmp_df.periodName.apply(lambda x: month_to_num[x])
    tmp_df['value'] = pd.to_numeric(tmp_df['value'], errors='coerce')
    tmp_df.rename(columns={'year': 'year', 'periodName': 'month', 'value': columns_BLS[table_index]}, inplace=True)
    tmp_df['Timeframe'] = pd.to_datetime(tmp_df[['year', 'month']].assign(day=1))
    tmp_df.drop(columns=['year', 'month'], inplace=True)
    grid.append(tmp_df)

final_BLS_df = grid[0]
for index in range(1, len(grid)):
    final_BLS_df = final_BLS_df.merge(grid[index], on='Timeframe', how='left')
    
final_BLS_df.set_index('Timeframe')
final_BLS_df.to_csv(os.path.join(data_folder, 'BLS_Data.csv'), index=False)

In [ ]:
url = 'https://api.bls.gov/publicAPI/v2/timeseries/data/'
columns_E = ['Civilian workers Total compensation', 'Private industry workers Total compensation', 'State and government workers Total compensation']
data = {
    "seriesid": ["CIU1010000000000A", "CIU2010000000000A", "CIU3010000000000A"],
    "startyear": "2005",
    "endyear": "2024",
    "catalog": True,
    "calculations": False,
    "annualaverage": False,
    "aspects": False,
    "registrationkey": "5974f3d1e0ac4efdad23d2e1b3e4b4e3"
}
html = r.post(url, json=data)
html = html.json()

grid = []

months = pd.date_range(start='01-01-2005', end='09-01-2024', freq='MS')
quarters = months.to_period('Q').astype(str)
month_df = pd.DataFrame({'Timeframe': months, 'quarter': quarters})
month_df['quarter'] = month_df['quarter'].apply(lambda x: x[len(x) - 1])
month_df['year'] = month_df['Timeframe'].apply(lambda x: x.year)



for table_index in range(0, len(columns_E)) :
    data = html['Results']['series'][table_index]['data']
    tmp_df = pd.DataFrame(data, columns=['year', 'periodName', 'value'])
    tmp_df['value'] = pd.to_numeric(tmp_df['value'], errors='coerce')
    tmp_df['year'] = pd.to_numeric(tmp_df['year'], errors='coerce')
    tmp_df.rename(columns={'year': 'year', 'periodName': 'quarter', 'value': columns_E[table_index]}, inplace=True)
    tmp_df['quarter'] = tmp_df['quarter'].apply(lambda x: x[0])
    tmp_df = pd.merge(month_df, tmp_df, how='left', on=['quarter', 'year'])
    tmp_df.drop(columns=['year', 'quarter'], inplace=True)
    grid.append(tmp_df)
    
final_BLS_e = grid[0]
for index in range(1, len(grid)):
    final_BLS_e = final_BLS_e.merge(grid[index], on='Timeframe', how='left')
    
final_BLS_e.to_csv(os.path.join(data_folder, 'BLS_salary.csv'), index=False)

## Personal Consumption Expenditures 

In [ ]:
from datetime import datetime
url = 'https://apps.bea.gov/api/data?&UserID=6E42AF4F-01C3-4ADB-9244-2DD195EFF66E&method=GetData&DataSetName=NIPA&TableName=T20805&Frequency=M&Year=2005,2006,2007,2008,2009,2010,2011,2012,2013,2014,2015,2016,2017,2018,2019,2020,2021,2022,2023,2024&ResultFormat=JSON'

html = r.get(url)
html = html.json()
food_pce = []
for table in html['BEAAPI']["Results"]["Data"]:
    if "Food" in table['LineDescription'] :
        food_pce.append(table)

df_pce = pd.DataFrame(food_pce, columns=['LineDescription', 'TimePeriod', 'DataValue'])
time_data = []
for time in df_pce['TimePeriod']:
    tmp = time.split('M')
    year = int(tmp[0])
    month = int(tmp[1])
    time_data.append(datetime(year, month, 1))

df_pce['TimePeriod'] = time_data
df_pce['DataValue'] = pd.to_numeric(df_pce['DataValue'].str.replace(',',''), errors='coerce')
df_pce.rename(columns={'DataValue': 'PCE', 'LineDescription': 'PCE Category', 'TimePeriod': 'date'}, inplace=True)
df_pce = df_pce.pivot_table(index='date', columns='PCE Category', values='PCE')
df_pce.to_csv(os.path.join(data_folder, 'pce.csv'))


In [ ]:
px.line(df_pce)

## USDA Price Received Index

In [ ]:
program = 'SURVEY'
sector = ['ANIMALS & PRODUCTS', 'CROPS']
animal_group = ['AQUACULTURE', 'DAIRY', 'LIVESTOCK', 'POULTRY']
crop_group = ['FIELD CROPS', 'FRUIT & TREE NUTS', 'HORTICULTURE', 'VEGETABLES']
animal_commodity = ['FOOD FISH', 'DAIRY PRODUCT TOTALS', 'LIVESTOCK TOTALS', 'POULTRY TOTALS']
crops_commodity = ['FIELD CROPS, OTHER', 'FRUIT & TREE NUT TOTALS', 'HORTICULTURE TOTALS', 'VEGETABLE TOTALS']
category = 'INDEX FOR PRICE RECEIVED, 2011'


In [ ]:
base_url = 'https://quickstats.nass.usda.gov/api/api_GET'
data_frames = []

def usda_request(param) :
    response = r.get(base_url, params=param)

    json = response.json()
    data = json['data']

    df = pd.DataFrame(data, columns=['commodity_desc', 'reference_period_desc', 'year', 'Value'])
    df.rename(columns={'commodity_desc': 'Commodity', 'reference_period_desc': 'month', 'year': 'year', 'Value': 'RPI'}, inplace=True)
    data_frames.append(df)
    
    

for index in [0, 1]:
    if index == 0 :
        for commodity in range(len(animal_group)):
            param = {
                'key': 'E621BE9B-36D0-3FF0-9333-443D794A932D',
                'source_desc': program,
                'sector_desc': sector[index],
                'group_desc': animal_group[commodity],
                'commodity_desc': animal_commodity[commodity],
                'statisticcat_desc': 'INDEX FOR PRICE RECEIVED, 2011',
                'freq_desc' : 'MONTHLY'
            }
            usda_request(param)
    else:
        for commodity in range(len(crop_group)):
            params = {
                'key': 'E621BE9B-36D0-3FF0-9333-443D794A932D',
                'source_desc': program,
                'sector_desc': sector[index],
                'group_desc': crop_group[commodity],
                'commodity_desc': crops_commodity[commodity],
                'statisticcat_desc': 'INDEX FOR PRICE RECEIVED, 2011',
                'freq_desc' : 'MONTHLY'
            }
            usda_request(params)

    # Make a request with these parameters


df_price_received = pd.concat(data_frames)

In [ ]:
month_abbr_to_num = {
    'JAN': 1,
    'FEB': 2,
    'MAR': 3,
    'APR': 4,
    'MAY': 5,
    'JUN': 6,
    'JUL': 7,
    'AUG': 8,
    'SEP': 9,
    'OCT': 10,
    'NOV': 11,
    'DEC': 12
}
df_price_received['month'] = df_price_received['month'].apply(lambda x: month_abbr_to_num[x]) 
df_price_received['RPI'] = pd.to_numeric(df_price_received['RPI'])
df_price_received['Timeframe'] = pd.to_datetime(df_price_received[['month', 'year']].assign(day=1))
df_price_received = df_price_received[df_price_received['Timeframe'] >= '2005-01-01']
df_price_received = df_price_received.pivot_table(index='Timeframe', columns='Commodity', values='RPI')
df_price_received.to_csv(os.path.join(data_folder, 'RPI.csv'))

### Combine Dataset

In [5]:
PPI_CPI_data = pd.read_csv(os.path.join(data_folder, 'BLS_Data.csv'))
salary_data = pd.read_csv(os.path.join(data_folder, 'BLS_salary.csv'))
df_price_received = pd.read_csv(os.path.join(data_folder, 'RPI.csv'))

In [6]:
final = pd.merge(PPI_CPI_data, salary_data, on='Timeframe', how='left').merge(df_price_received, on='Timeframe')

In [7]:
final.to_csv(os.path.join(data_folder, 'final.csv'), index=False)